In [99]:
encryped_flag = b'\x5b\x1e\x56\x4b\x6e\x41\x5c\x0e\x39\x4e\x04\x01\x38\x4b\x08\x55\x3a\x4e\x5c\x59\x7b\x6d\x4a\x5c\x5a\x68\x4d\x50\x01\x3d\x6e\x4b'

# flag = bytes(a ^ b for (a, b) in zip(encryped_flag, zero_response))
print(len(encryped_flag))

32


Length of flag is 16 - the next encryption will start from offset 16 in key
The key has length of 50000

Idea: give 50000 zeros
* when the server starts it will encrypt the flag with the first 16 bytes of the key
* the second attempt will use the OTP flag from offset 16
* If we give an array of zeros the length of the key, the first 50000-16 bytes will be the remaining bytes in the key file, but crucially the last 16 bytes of the output will be the first 16 bytes of the key file!!!!

In [122]:
import socket

HOST = "mercury.picoctf.net"
PORT = 20266

KEY_LEN = 50000
FLAG_LEN = 32


with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    flag = (s.recv(128) + s.recv(256)).decode()
    print(flag)

    crafted_message = (b"\x00" * (KEY_LEN - FLAG_LEN)) + b"\n"
    s.sendall(crafted_message)

    print(s.recv(12))
    otp = b""
    while len(otp) < ((KEY_LEN - FLAG_LEN) * 2):
        otp += s.recv(1)
        # print(len(otp))
    # key = otp[-(FLAG_LEN + 1):-1]
    print(len(otp))

    print(s.recv(128))
    # print(s.recv(128))

    with open("key", "w") as key_file:
        key_file.write(otp.decode())

    s.sendall(b"\x00" * FLAG_LEN + B"\n")
    key = s.recv(12 + FLAG_LEN * 2)
    print(key)
    # print(otp[-(FLAG_LEN * 2):].decode())


******************Welcome to our OTP implementation!******************
This is the encrypted flag!
5b1e564b6e415c0e394e0401384b08553a4e5c597b6d4a5c5a684d50013d6e4b

What data would you like to encrypt? 
b'Here ya go!\n'
99936
b'\n\nWhat data would you like to encrypt? '
b'Here ya go!\n6227667c5c7865385c7862365c7831625c7839384b5c7864385c7861365e5c78'


In [123]:
# zero_response = b"\x78\x39\x63\x5c\x78\x30\x34\x5c\x78\x61\x36\x5c\x78\x39\x39\x5c\x78\x65\x38\x4c\x5c\x78\x39\x39\x5c\x78\x31\x33\x5c\x78\x39\x33"
# zero_response = b"\x5c\x78\x38\x30\x5c\x78\x61\x32\x5c\x78\x31\x63\x5f\x3e\x61\x5c\x78\x31\x39\x5c\x78\x38\x33\x5a\x68\x52\x5c\x78\x39\x66\x5c\x78"
e_flag = "5b1e564b6e415c0e394e0401384b08553a4e5c597b6d4a5c5a684d50013d6e4b"
# zero_response = b"\x5c\x78\x31\x34\x5c\x78\x38\x61\x5c\x78\x66\x62\x60\x5c\x78\x38\x30\x5c\x78\x31\x64\x5c\x78\x38\x36\x5c\x78\x39\x39\x5c\x78\x64"
# zero_response = b"\x62\x27\x66\x7c\x5c\x78\x65\x38\x5c\x78\x62\x36\x5c\x78\x31\x62\x5c\x78\x39\x38\x4b\x5c\x78\x64\x38\x5c"
zero_response = b"\x62\x27\x66\x7c\x5c\x78\x65\x38\x5c\x78\x62\x36\x5c\x78\x31\x62\x5c\x78\x39\x38\x4b\x5c\x78\x64\x38\x5c\x78\x61\x36\x5e\x5c\x78"

print(f"Encrypted flag: {encryped_flag.decode()}")
print(f"Zero response: {zero_response.decode()}")

flag = bytes(a ^ b for (a, b) in zip(encryped_flag, zero_response))
print(f"Flag: picoCTF{{{flag.decode()}}}")

Encrypted flag: [VKnA\9N8U:N\Y{mJ\ZhMP=nK
Zero response: b'f|\xe8\xb6\x1b\x98K\xd8\xa6^\x
Flag: picoCTF{99072996e6f7d397f6ea0128b4517c23}


In [112]:
# test_f_e = b"\x5b\x1e\x56\x4b\x6e\x41\x5c\x0e\x39\x4e\x04\x01\x38\x4b\x08\x55\x3a\x4e\x5c\x59\x7b\x6d\x4a\x5c\x5a\x68\x4d\x50\x01\x3d\x6e\x4b"
test_f_e = b"\x50\x04\x6f\x4c\x09\x54\x6f\x4c\x50\x03\x6f\x4c\x55\x56\x6f\x4c\x09\x51\x6f\x4c\x52\x07\x55\x06\x6d\x4a\x56\x01\x1c\x01\x6f\x4c"

test_key = b"\x61\x36\x5c\x78\x38\x66\x5c\x78\x61\x31\x5c\x78\x64\x64\x5c\x78\x38\x63\x5c\x78\x63\x35\x66\x32\x5c\x78\x65\x35\x2d\x33\x5c\x78"

flag = bytes(a ^ b for (a, b) in zip(test_f_e, test_key))
print(f"Flag: picoCTF{{{flag.decode()}}}")

Flag: picoCTF{12341234123412341234123412341234}
